In [1]:
import pandas as pd
from datetime import datetime, timedelta
from analyzer.live_entry_strats import LiveEntryStrats as les
from analyzer.live_exit_strats import LiveExitStrats as lxs
from time import sleep
from database.comet import Comet
from coinbase.coinbase import Coinbase as cbs
from processor.processor import Processor as p
import pytz
import math

In [2]:
comet = Comet()

In [6]:
comet.cloud_connect()
historicals = comet.retrieve("cloud_test_coinbase_hourly")
trading_params = comet.retrieve("btc_trading_params")
backtest_results = comet.retrieve("btc_backtest_results")
comet.disconnect()

In [4]:
# backtest_results.sort_values("pv",ascending=False)

,signal,req,trades,pv,days,retrack_days,value,conservative,entry_strategy,exit_strategy
2592,0.2,0.3,4,285.610000,35.750000,7,False,True,signal_based,hold
3552,0.2,0.3,4,285.610000,35.500000,7,False,True,all,hold
2140,0.2,0.1,11,285.311671,12.272727,14,False,True,standard,hold
193,0.2,0.4,3,274.400000,73.333333,7,True,True,standard,hold
4033,0.2,0.4,3,274.400000,73.333333,7,True,False,standard,hold
...,...,...,...,...,...,...,...,...,...,...
6243,0.0,0.4,12,69.781993,28.000000,1,False,False,signal_based,due_date
2463,0.0,0.4,11,69.781993,29.272727,7,False,True,signal_based,due_date
2464,0.0,0.5,11,69.781993,29.272727,7,False,True,signal_based,due_date
2640,0.0,0.1,12,69.781993,28.000000,1,False,True,signal_based,adaptive_due_date


In [8]:
# optimal_entries = []
# for row in historicals.iterrows():
#     data = row[1]
#     signal = data["signal"]
#     value = signal > 0
#     signal_parameter = math.floor(abs(signal) % 10)
#     retrack_days = trading_params["retrack_days"].item()
#     optimal_entry = backtest_results[(backtest_results["signal"]==signal_parameter)
#                                     & (backtest_results["retrack_days"]==retrack_days)
#                                      & (backtest_results["value"]==value)
#                                      & (backtest_results["pv"]>100)
#                                      & (backtest_results["days"]<30)
#                                     ].sort_values("pv",ascending=False).iloc[0]
#     optimal_entries.append(optimal_entry)

In [9]:
pd.DataFrame(optimal_entries)

,signal,req,trades,pv,days,retrack_days,value,conservative,entry_strategy,exit_strategy
3390,0.0,0.1,12,123.366141,19.083333,3,False,True,all,due_date
3390,0.0,0.1,12,123.366141,19.083333,3,False,True,all,due_date
3390,0.0,0.1,12,123.366141,19.083333,3,False,True,all,due_date
3390,0.0,0.1,12,123.366141,19.083333,3,False,True,all,due_date
1470,0.0,0.1,11,128.709542,6.454545,3,True,True,all,due_date
1470,0.0,0.1,11,128.709542,6.454545,3,True,True,all,due_date
1470,0.0,0.1,11,128.709542,6.454545,3,True,True,all,due_date
1470,0.0,0.1,11,128.709542,6.454545,3,True,True,all,due_date
1470,0.0,0.1,11,128.709542,6.454545,3,True,True,all,due_date
1470,0.0,0.1,11,128.709542,6.454545,3,True,True,all,due_date


In [5]:
offerings = les.entry_analysis(
            trading_params["entry_strategy"].item()
           ,historicals
           ,trading_params["signal"].item()
           ,trading_params["value"].item()
           ,trading_params["conservative"].item()
            )

In [10]:
accounts = cbs.get_accounts()
balance = accounts[accounts["currency"]=="USD"]["balance"].iloc[0]
pending_orders = cbs.get_orders()
pending_orders = p.live_column_date_processing(pending_orders.rename(columns={"created_at":"date"}))
fls = []
status = "fills"
for currency in accounts["currency"].unique():
    fill = cbs.get_fill(currency)
    if len(fill) > 0:
        try:
            f = pd.DataFrame(fill)
            fls.append(f)
        except:
            continue
## sells
if len(fls) > 0:
    fills = pd.concat(fls)
    #store_non_existing_executed_buy
    existing_fills = comet.retrieve_fills()
    if existing_fills.index.size > 1:
        existing_order_ids = list(existing_fills["order_id"])
    else:
        existing_order_ids = []
    new_fills = fills[~fills["order_id"].isin(existing_order_ids)]        
    if new_fills.index.size > 0:
        incomplete_trades = new_fills[(new_fills["side"]=="buy")]
        incomplete_trades["size"] = [float(x) for x in incomplete_trades["size"]]
        incomplete_trades["price"] = [float(x) for x in incomplete_trades["price"]]
        for oi in incomplete_trades["order_id"].unique():
            order_trades = incomplete_trades[incomplete_trades["order_id"]==oi]
            if len([x for x in order_trades["settled"] if x == False]) == 0 and order_trades.index.size > 0:
                incomplete_trade = lxs.exit_analysis(exit_strategy,merged,order_trades,retrack_days,req)
                if "sell_price" in incomplete_trade.keys():
                    sell_statement = cbs.place_sell(incomplete_trade["product_id"]
                                                    ,incomplete_trade["sell_price"]
                                                    ,incomplete_trade["size"])
                    incomplete_trade["sell_id"] = sell_statement["id"]

In [13]:
if new_fills.index.size > 0:
    completed_trades = new_fills[(new_fills["side"]=="sell")]
    for soi in completed_trades["order_id"].unique():
        sell_order_trades = completed_trades[completed_trades["order_id"]==soi]
        if len([x for x in sell_order_trades["settled"] if x == False]) == 0:
            complete_trade = sell_order_trades.iloc[0]
            order_id = complete_trade["order_id"]
            one_half = comet.retrieve_incomplete_trade(order_id)
            one_half["sell_date"] = complete_trade["created_at"]
            one_half["sell_price"] = complete_trade["price"]

NameError: name 'incomplete_trades' is not defined

In [15]:
completed_trades
incomplete_trades

NameError: name 'completed_trades' is not defined